In [1]:
try:
    import google.colab
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
import torch
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Note: using Google CoLab
Using device: cuda


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau

names = ['year', 'month', 'day', 'dec_year', 'sn_value',
         'sn_error', 'obs_num', 'extra']
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/SN_d_tot_V2.0.csv",
    sep=';', header=None, names=names,
    na_values=['-1'], index_col=False)

In [3]:
df.fillna(0, inplace=True)

In [4]:
df.columns

Index(['year', 'month', 'day', 'dec_year', 'sn_value', 'sn_error', 'obs_num',
       'extra'],
      dtype='object')

In [5]:
df_values = df[['sn_value', 'sn_error', 'obs_num','extra']]

In [6]:
df_values

,sn_value,sn_error,obs_num,extra
0,-1,0.0,0,1
1,-1,0.0,0,1
2,-1,0.0,0,1
3,-1,0.0,0,1
4,-1,0.0,0,1
...,...,...,...,...
72860,21,1.1,25,0
72861,19,1.2,36,0
72862,17,1.1,22,0
72863,12,0.5,25,0


In [7]:
df_values['sn_value'] = df_values.sn_value.astype('float')

<ipython-input-7-584ba0fa553f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_values['sn_value'] = df_values.sn_value.astype('float')


In [8]:
# Check if the column contains any strings and print them

column_name = 'sn_value'

non_numeric_values = df_values[df_values[column_name].apply(lambda x: isinstance(x, str))][column_name]

if not non_numeric_values.empty:
    print(f"Non-numeric values in '{column_name}':")
    print(non_numeric_values)
else:
    print(f"No non-numeric values found in '{column_name}'.")

No non-numeric values found in 'sn_value'.


In [9]:
df_values['sn_value'] = df_values['sn_value'].astype(float)

<ipython-input-9-cec61cc3e9be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_values['sn_value'] = df_values['sn_value'].astype(float)


In [10]:
df_values.dtypes

,0
sn_value,float64
sn_error,float64
obs_num,int64
extra,int64


In [11]:
df_values

,sn_value,sn_error,obs_num,extra
0,-1.0,0.0,0,1
1,-1.0,0.0,0,1
2,-1.0,0.0,0,1
3,-1.0,0.0,0,1
4,-1.0,0.0,0,1
...,...,...,...,...
72860,21.0,1.1,25,0
72861,19.0,1.2,36,0
72862,17.0,1.1,22,0
72863,12.0,0.5,25,0


In [12]:
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_values), columns=df_values.columns)

In [13]:
pd.DataFrame(df_scaled)

,sn_value,sn_error,obs_num,extra
0,-1.045726,-1.502628,-0.572843,0.035362
1,-1.045726,-1.502628,-0.572843,0.035362
2,-1.045726,-1.502628,-0.572843,0.035362
3,-1.045726,-1.502628,-0.572843,0.035362
4,-1.045726,-1.502628,-0.572843,0.035362
...,...,...,...,...
72860,-0.761959,-1.267897,2.937699,-28.279220
72861,-0.787756,-1.246558,4.482338,-28.279220
72862,-0.813553,-1.267897,2.516434,-28.279220
72863,-0.878045,-1.395932,2.937699,-28.279220


In [14]:
df_train = df_scaled.iloc[:(72865-1000)]
df_test = df_scaled.iloc[(72865-1000):]

In [15]:
df_train

,sn_value,sn_error,obs_num,extra
0,-1.045726,-1.502628,-0.572843,0.035362
1,-1.045726,-1.502628,-0.572843,0.035362
2,-1.045726,-1.502628,-0.572843,0.035362
3,-1.045726,-1.502628,-0.572843,0.035362
4,-1.045726,-1.502628,-0.572843,0.035362
...,...,...,...,...
71860,1.263105,1.250128,2.235591,0.035362
71861,0.708469,0.204508,1.393061,0.035362
71862,0.592383,0.076473,1.393061,0.035362
71863,0.476296,0.183169,1.673904,0.035362


In [16]:
df_test

,sn_value,sn_error,obs_num,extra
71865,0.115139,0.076473,0.550531,0.035362
71866,-0.155730,-0.328972,1.252639,0.035362
71867,-0.155730,-0.478347,1.393061,0.035362
71868,-0.078339,0.631292,0.971796,0.035362
71869,-0.271816,-0.734417,1.814326,0.035362
...,...,...,...,...
72860,-0.761959,-1.267897,2.937699,-28.279220
72861,-0.787756,-1.246558,4.482338,-28.279220
72862,-0.813553,-1.267897,2.516434,-28.279220
72863,-0.878045,-1.395932,2.937699,-28.279220


In [17]:
def split_series(x_series, y_series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations
  #
  X, y = list(), list()
  for window_start in range(len(x_series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(x_series):
      break
    # slicing the past and future parts of the window
    past = x_series[window_start:past_end,:]

  for window_start in range(len(y_series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(y_series):
      break
    # slicing the past and future parts of the window
    future = y_series[past_end:future_end]


    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [18]:
n_past = 200
n_future = 5
n_features = 4

In [19]:
x_train, y_train = split_series(df_train.values,df_train.sn_value.values, n_past, n_future)

In [20]:
x_test, y_test = split_series(df_test.values,df_test.sn_value.values, n_past, n_future)

In [21]:
# # Data Preprocessing
# start_id = max(df[df['obs_num'] == 0].index.tolist()) + 1
# df = df[start_id:].copy()
# df['sn_value'] = df['sn_value'].astype(float)
# df_train = df[df['year'] < 2000]
# df_test = df[df['year'] >= 2000]

# spots_train = df_train['sn_value'].to_numpy().reshape(-1, 1)
# spots_test = df_test['sn_value'].to_numpy().reshape(-1, 1)

# scaler = StandardScaler()
# spots_train = scaler.fit_transform(spots_train).flatten().tolist()
# spots_test = scaler.transform(spots_test).flatten().tolist()



Just like we did for LSTM in the previous section, we again break the data into sequences.

In [22]:
# # Sequence Data Preparation
# SEQUENCE_SIZE = 10

# def to_sequences(seq_size, obs):
#     x = []
#     y = []
#     for i in range(len(obs) - seq_size):
#         window = obs[i:(i + seq_size)]
#         after_window = obs[i + seq_size]
#         x.append(window)
#         y.append(after_window)
#     return torch.tensor(x, dtype=torch.float32).view(-1, seq_size, 1), torch.tensor(y, dtype=torch.float32).view(-1, 1)

# x_train, y_train = to_sequences(SEQUENCE_SIZE, spots_train)
# x_test, y_test = to_sequences(SEQUENCE_SIZE, spots_test)

# # Setup data loaders for batch
# train_dataset = TensorDataset(x_train, y_train)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# test_dataset = TensorDataset(x_test, y_test)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [23]:
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
x_test  = torch.tensor(x_test, dtype=torch.float32)
y_test  = torch.tensor(y_test, dtype=torch.float32)

In [24]:
# Setup data loaders for batch
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Position Encoding for Transformers

In the realm of the transformer architecture, a pivotal component that ensures the model's success is its ability to consider the sequence's order. Unlike traditional RNNs or LSTMs, which process sequences step-by-step and inherently respect their order, transformers process all tokens in a sequence simultaneously. While this parallel processing significantly boosts computational efficiency and allows for long-range dependencies to be captured more effectively, it also means that transformers, in their native form, are oblivious to the position or order of tokens in a sequence. This is where the concept of positional encoding comes into play.

Positional encoding is a mechanism to provide the transformer with information about the position of tokens within a sequence. Essentially, it infuses order information into the otherwise position-agnostic embeddings. By adding positional encodings to the token embeddings before feeding them into the transformer, each token's position in the sequence becomes discernible to the model.

Positional encodings are vectors that get added to the embeddings of tokens. The intuition is to design these vectors in such a way that their values or patterns are unique for each position, allowing the model to differentiate between different positions in the sequence.

A popular method to generate positional encodings is using sinusoidal functions. For a given position $p$ in the sequence and dimension $d$ of the embedding, the positional encoding is computed as:

$$ PE(2,i) = \sin(\frac{p}{10000^{2i/d}}) $$
$$ PE(2,i+1) = \cos(\frac{p}{10000^{2i/d}}) $$

Where $i$ is the dimension index. These sinusoidal functions generate values between -1 and 1 and ensure a unique and repeatable pattern for each position.

The choice of sinusoidal functions isn't arbitrary. They have two compelling properties:

1. They produce values between -1 and 1, making them compatible with most embedding value ranges.
2. Their patterns allow the model to extrapolate positions beyond the sequence lengths seen during training.

One might wonder, why not just append or add the token's position as an integer to the embedding? The challenge with this approach is scale. Embedding values, especially after being trained, can exist within a specific range, and directly adding large integers (for tokens further down in long sequences) might disrupt the information in the original embeddings.

Furthermore, using raw integers wouldn't provide a consistent way for the model to generalize or extrapolate to sequence lengths not seen during training. Sinusoidal functions, in contrast, offer a predictable pattern that aids in such extrapolation.

The following code describes a simple implementation of a transformer-based model using PyTorch's built-in functionalities. The **TransformerModel** class encapsulates a transformer-based neural network designed for sequence processing. Upon initialization, the model sets up several components: an encoder to adjust the input data to a desired dimension, a **pos_encoder** to bestow the sequence with positional information, a core **transformer_encoder** comprising several layers to process the sequence, and a **decoder** to produce the final output. As data flows through the model during the forward pass, it undergoes a series of transformations: it's first projected to a higher dimension, then augmented with positional encodings, processed by the transformer layers, and finally, the last token's representation is harnessed to produce the output. An instance of this model is readily created and can be assigned to a computation device for further training or inference.



In [25]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

# Constructing the Transformer Model

The following code constructs the actual transformer-based model for time series prediction. The model is constructed to accept the following parameters.

* **input_dim**: The dimension of the input data, in this case we use only one input, the number of sunspots.
* **d_model**: The number of features in the transformer model's internal representations (also the size of embeddings). This controls how much a model can remember and process.
* **nhead**: The number of attention heads in the multi-head self-attention mechanism.
* **num_layers**: The number of transformer encoder layers.
dropout: The dropout probability.



In [26]:
# Model definition using Transformer
class TransformerModel(nn.Module):
    def __init__(self, input_dim=4, d_model=64, nhead=4, num_layers=2, dropout=0.2):
        super(TransformerModel, self).__init__()

        self.encoder = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, 5)

    def forward(self, x):
        x = self.encoder(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = self.decoder(x[:, -1, :])
        return x

model = TransformerModel().to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


The Transformer architecture in PyTorch is governed by crucial configuration choices, among which **d_model**, **nhead**, and **num_layers** hold significant weight. The **d_model** denotes the dimensionality of the input embeddings and affects the model's capacity to learn intricate representations. While a more substantial **d_model** can bolster the richness of the model's understanding, it also amplifies the computational demand and can pose overfitting risks if not carefully chosen. Parallelly, the model's gradient flow and initialization are impacted by this choice, though the Transformer's normalization layers often moderate potential issues.

On the other hand, **nhead** reflects the count of heads in the multi-head attention mechanism. A higher number of heads grants the model the prowess to simultaneously focus on diverse segments of the input, enabling the capture of varied contextual nuances. However, there's a trade-off. Beyond a specific threshold, the computational overhead might outweigh the marginal performance gains. This parallel processing, provided by multiple attention heads, tends to offer more stable and varied gradient information, positively influencing the training dynamics.

Lastly, the **num_layers** parameter dictates the depth of the Transformer, determining the number of stacked encoder or decoder layers. A deeper model, as a result of increased layers, can discern more complex and hierarchical relationships in data. Still, there's a caveat: after a certain depth, potential performance enhancements may plateau, and the risk of overfitting might escalate. Training deeper models also comes with its set of challenges. Although residual connections and normalization in Transformers alleviate some concerns, a high layer count might necessitate techniques like gradient clipping or learning rate adjustments for stable training.

In essence, these parameters intricately balance model capacity, computational efficiency, and generalization capability. Their optimal settings often emerge from task-specific experimentation, the nature of the data, and available computational prowess.

## Training the Model


Training a transformer-based model adheres to many of the familiar paradigms and best practices that apply to other neural network architectures. Much like the models we've encountered before, a transformer-based model benefits from training in batches, which helps in both computational efficiency and generalization. Batched training ensures that the model updates its weights based on the average gradient over several data points, rather than being excessively influenced by any single instance. Additionally, the use of early stopping acts as a safeguard against overfitting. By monitoring the model's performance on a validation set and halting training when no significant improvement is observed over a set number of epochs, we ensure that the model generalizes well and doesn't just memorize the training data. The validation set, it remains an essential component in the training regimen, providing a proxy measure of the model's performance on unseen data and guiding hyperparameter tuning. Thus, while transformer architectures introduce novel mechanisms and complexities, the foundational principles of training deep learning models in PyTorch remain consistent.

In [27]:
# Train the model
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True)

epochs = 10
early_stop_count = 0
min_val_loss = float('inf')

for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for batch in test_loader:
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            val_losses.append(loss.item())

    val_loss = np.mean(val_losses)
    scheduler.step(val_loss)

    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1

    if early_stop_count >= 5:
        print("Early stopping!")
        break
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss:.4f}")



/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/10, Validation Loss: 0.3859
Epoch 2/10, Validation Loss: 0.4307
Epoch 3/10, Validation Loss: 0.3825
Epoch 4/10, Validation Loss: 0.3730
Epoch 5/10, Validation Loss: 0.3695
Epoch 6/10, Validation Loss: 0.3750
Epoch 7/10, Validation Loss: 0.3636
Epoch 8/10, Validation Loss: 0.3605
Epoch 9/10, Validation Loss: 0.3899
Epoch 10/10, Validation Loss: 0.3598


We can now evaluate the performance of this model.

In [29]:
# Evaluation
model.eval()
predictions = []
with torch.no_grad():
    for batch in test_loader:
        x_batch, y_batch = batch
        x_batch = x_batch.to(device)
        outputs = model(x_batch)
        predictions.extend(outputs.squeeze().tolist())

# rmse =
# np.sqrt(np.mean((scaler.inverse_transform(np.array(predictions).reshape(-1,
# 1)) - scaler.inverse_transform(y_test.numpy().reshape(-1, 1)))**2))
# print(f"Score (RMSE): {rmse:.4f}")

In [41]:
scaler.inverse_transform(np.array([predictions[0]]))

ValueError: operands could not be broadcast together with shapes (1,5) (4,) (1,5) 

In [40]:
np.array([predictions[0]])

array([[-0.00693802, -0.00180639, -0.00835184, -0.01171439, -0.00708179]])